In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np

In [2]:
flights_df = pd.read_csv('Datasets/Merged_Flights.csv')
flights_df.shape

F:\WIN temp files\ipykernel_12684\2559509712.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv('Datasets/Merged_Flights.csv')


(26490448, 18)

In [3]:
flights_df.drop_duplicates(subset=['icao24', 'last_position'], inplace = True)
flights_df = flights_df[['timestamp', 'icao24', 'latitude', 'longitude', 'ground_speed', 'track', 'vertical_rate', 'baro_altitude']]
flights_df.shape

(19486764, 8)

In [4]:

flights_df = flights_df[(flights_df['longitude'] <= 20) & (flights_df['longitude'] >= -20)]
flights_df = flights_df[(flights_df['baro_altitude'] >= 10000) & (flights_df['baro_altitude'] <= 50000)]
flights_df = flights_df[(flights_df['vertical_rate'] >= -4500) & (flights_df['vertical_rate'] <= 4500)]
flights_df.shape

(15274354, 8)

In [5]:
# Convert 'timestamp' and 'last_position' to datetime
flights_df['timestamp'] = pd.to_datetime(flights_df['timestamp'])
flights_df.shape

(15274354, 8)

In [6]:
flights_df.isnull().sum(), flights_df.shape

(timestamp        0
 icao24           0
 latitude         0
 longitude        0
 ground_speed     0
 track            0
 vertical_rate    0
 baro_altitude    0
 dtype: int64,
 (15274354, 8))

In [7]:
flights_df.dropna(inplace = True)
flights_df.isnull().sum(), flights_df.shape

(timestamp        0
 icao24           0
 latitude         0
 longitude        0
 ground_speed     0
 track            0
 vertical_rate    0
 baro_altitude    0
 dtype: int64,
 (15274354, 8))

In [8]:
flights_df_sorted = flights_df.sort_values(by=['icao24', 'timestamp'])
flights_df_sorted.head(10000)

,timestamp,icao24,latitude,longitude,ground_speed,track,vertical_rate,baro_altitude
10060824,2023-05-23 09:00:02,342582,41.280988,2.399389,268.463918,84.013336,1088.0,10025.0
10060825,2023-05-23 09:00:03,342582,41.281128,2.400887,269.673644,83.612883,1088.0,10050.0
10060826,2023-05-23 09:00:04,342582,41.281314,2.402697,270.896425,83.216026,1152.0,10075.0
10060827,2023-05-23 09:00:05,342582,41.281454,2.404570,273.124278,82.848964,1152.0,10075.0
10060828,2023-05-23 09:00:06,342582,41.281547,2.405444,273.380451,82.433056,1216.0,10100.0
...,...,...,...,...,...,...,...,...
9551848,2023-05-13 14:53:48,343194,49.277313,1.607112,410.077687,29.190312,64.0,36025.0
9551849,2023-05-13 14:53:49,343194,49.279408,1.608919,409.694655,29.380608,64.0,36025.0
9551850,2023-05-13 14:53:50,343194,49.281097,1.610413,409.810578,29.692857,64.0,36025.0
9551851,2023-05-13 14:53:51,343194,49.282654,1.611751,410.306848,29.814163,64.0,36025.0


In [9]:
# Downsampling the dataset by keeping one in every 60 data points
downsampled_df = flights_df_sorted.iloc[::120, :]
downsampled_df.shape

(127287, 8)

In [10]:
# Function to find the row with the closest timestamp to the target time
def find_first_future_row(df, target_time, icao24):
    future_rows = df[(df['icao24'] == icao24) & (df['timestamp'] >= target_time) & (df['timestamp'] <= target_time + timedelta(minutes=2))]
    return future_rows.iloc[0] if not future_rows.empty else None

# Testing the function on a small subset of the data
subset_df = downsampled_df

# Adding new columns for future position and altitude
subset_df['latitude_in_10min'] = np.nan
subset_df['longitude_in_10min'] = np.nan
subset_df['baro_altitude_in_10min'] = np.nan

rows = subset_df.shape[0]
n=0
# Iterating over each row in the subset
for index, row in subset_df.iterrows():
    future_time = row['timestamp'] + timedelta(minutes=10)
    future_row = find_first_future_row(subset_df, future_time, row['icao24'])
    if future_row is not None:
        subset_df.at[index, 'latitude_in_10min'] = future_row['latitude']
        subset_df.at[index, 'longitude_in_10min'] = future_row['longitude']
        subset_df.at[index, 'baro_altitude_in_10min'] = future_row['baro_altitude']
    n += 1
    if n % 1000 == 0:
        print(round((n/rows)*100,1), '%')



# Displaying the modified subset to check the results
subset_df.head()
subset_df.isna().sum()

F:\WIN temp files\ipykernel_12684\2624164120.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['latitude_in_10min'] = np.nan
F:\WIN temp files\ipykernel_12684\2624164120.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['longitude_in_10min'] = np.nan
F:\WIN temp files\ipykernel_12684\2624164120.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

0.8 %
1.6 %
2.4 %
3.1 %
3.9 %
4.7 %
5.5 %
6.3 %
7.1 %
7.9 %
8.6 %
9.4 %
10.2 %
11.0 %
11.8 %
12.6 %
13.4 %
14.1 %
14.9 %
15.7 %
16.5 %
17.3 %
18.1 %
18.9 %
19.6 %
20.4 %
21.2 %
22.0 %
22.8 %
23.6 %
24.4 %
25.1 %
25.9 %
26.7 %
27.5 %
28.3 %
29.1 %
29.9 %
30.6 %
31.4 %
32.2 %
33.0 %
33.8 %
34.6 %
35.4 %
36.1 %
36.9 %
37.7 %
38.5 %
39.3 %
40.1 %
40.9 %
41.6 %
42.4 %
43.2 %
44.0 %
44.8 %
45.6 %
46.4 %
47.1 %
47.9 %
48.7 %
49.5 %
50.3 %
51.1 %
51.9 %
52.6 %
53.4 %
54.2 %
55.0 %
55.8 %
56.6 %
57.4 %
58.1 %
58.9 %
59.7 %
60.5 %
61.3 %
62.1 %
62.9 %
63.6 %
64.4 %
65.2 %
66.0 %
66.8 %
67.6 %
68.3 %
69.1 %
69.9 %
70.7 %
71.5 %
72.3 %
73.1 %
73.8 %
74.6 %
75.4 %
76.2 %
77.0 %
77.8 %
78.6 %
79.3 %
80.1 %
80.9 %
81.7 %
82.5 %
83.3 %
84.1 %
84.8 %
85.6 %
86.4 %
87.2 %
88.0 %
88.8 %
89.6 %
90.3 %
91.1 %
91.9 %
92.7 %
93.5 %
94.3 %
95.1 %
95.8 %
96.6 %
97.4 %
98.2 %
99.0 %
99.8 %


timestamp                     0
icao24                        0
latitude                      0
longitude                     0
ground_speed                  0
track                         0
vertical_rate                 0
baro_altitude                 0
latitude_in_10min         17451
longitude_in_10min        17451
baro_altitude_in_10min    17451
dtype: int64

In [11]:
subset_df.dropna(inplace = True)

F:\WIN temp files\ipykernel_12684\1856307613.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df.dropna(inplace = True)


In [12]:
subset_df['Climbing'] = subset_df['vertical_rate'] > 0
subset_df['Descending'] = subset_df['vertical_rate'] < 0
subset_df['Cruise'] = subset_df['vertical_rate'] == 0

F:\WIN temp files\ipykernel_12684\1686109439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['Climbing'] = subset_df['vertical_rate'] > 0
F:\WIN temp files\ipykernel_12684\1686109439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['Descending'] = subset_df['vertical_rate'] < 0
F:\WIN temp files\ipykernel_12684\1686109439.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [13]:
subset_df

,timestamp,icao24,latitude,longitude,ground_speed,track,vertical_rate,baro_altitude,latitude_in_10min,longitude_in_10min,baro_altitude_in_10min,Climbing,Descending,Cruise
10060824,2023-05-23 09:00:02,342582,41.280988,2.399389,268.463918,84.013336,1088.0,10025.0,42.024148,2.001172,29175.0,True,False,False
10060944,2023-05-23 09:02:02,342582,41.342204,2.601513,330.287468,48.928307,3136.0,14625.0,42.228513,1.819898,31900.0,True,False,False
10061064,2023-05-23 09:04:02,342582,41.502949,2.601460,372.413427,322.637777,896.0,19225.0,42.470910,1.768352,34800.0,True,False,False
10061184,2023-05-23 09:06:02,342582,41.672353,2.411818,415.895060,318.606717,1472.0,22675.0,42.707428,1.718674,36000.0,True,False,False
10061304,2023-05-23 09:08:02,342582,41.843216,2.212996,411.095741,318.945186,1728.0,27100.0,42.946115,1.693072,36000.0,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20942036,2023-08-27 12:15:59,ad5b90,53.724976,-9.134426,429.283917,277.091907,64.0,36000.0,53.857215,-11.247011,40000.0,True,False,False
20942194,2023-08-27 12:18:37,ad5b90,53.761690,-9.662314,428.923865,276.694229,-64.0,36000.0,53.878647,-11.658927,40000.0,False,True,False
20942315,2023-08-27 12:20:38,ad5b90,53.787048,-10.055158,408.485877,276.324702,64.0,36000.0,53.898926,-12.067674,40000.0,True,False,False
20942435,2023-08-27 12:22:38,ad5b90,53.811454,-10.442020,422.403470,276.115504,1024.0,37675.0,53.917694,-12.492711,40000.0,True,False,False


In [14]:
subset_df.to_csv('Datasets/Cleaned_prepared_data.csv')